## 학습 알고리즘

- 유저의 행동데이터를 받아와서 기초 알고리즘의 가중치들 수정
- 메인 알고리즘을 작성하여 유저들의 속성과 그 행동 데이터들 만으로 가중치 설정
    - 유저의 향후 행동 데이터를 계속 학습

### 행동 데이터 (유저 선택의 결과)

- 유저가 일정을 변경하는 버튼을 따로 만들고, 이 버튼(일정 확정)이 푸시 되었을 때와 그 전을 비교해서 가중치에 수정을 가한다
- 유저가 자유롭게 일정을 드래그 & 드롭으로 수정할 수 있게 만들고, 수정된 것에 대한 가중치를 수집한다 (이런 경우엔 세션 종료 시 변경 여부를 가져와서 가중치 수정을 가하는 방법이겠지 ??)
- todo를 루틴과 루틴이 아닌 것으로 구분

- 유저가 많이 수행하는 todo의 카테고리를 잡고, 해당 카테고리에서 벗어나는 카테고리를 주의 카테고리로 설정